In [ ]:
from __future__ import print_function

import glob, os, random, torch, shutil, cv2, h5py, joblib, gc, pickle, timm, concurrent.futures, json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl

from openslide import OpenSlide
from PIL import Image, ImageEnhance, ImageOps, ImageDraw
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from pathlib import Path
from pprint import pprint
from tempfile import TemporaryDirectory
from sklearn.metrics import precision_recall_curve, confusion_matrix
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, average_precision_score
from sklearn.preprocessing import OneHotEncoder

from ultralytics import YOLO

os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

seed = 0

%matplotlib inline

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [ ]:
# クラス名の辞書
class_names = {
    0: "Glo",
    1: "GS",
    2: "Cres",
    3: "SegSc"
}

In [ ]:
d = 'base_PATH'
models = sorted(glob.glob(d + '/PATH/*.pt'))
models

In [ ]:
files = sorted(glob.glob(d + '/PATH/*/*.png'))
print(len(files))
print(files[0])

In [ ]:
def yolo_predict(image_patch, is_top_edge=False, is_left_edge=False, is_bottom_edge=False, is_right_edge=False, YOLO_threshold=0.01, iou_threshold=0.5, image_size=640, edge_margin=10):
    # YOLOモデルによる予測
    results = model.predict(image_patch, save=False, conf=YOLO_threshold)
    result_object = results[0]
    bounding_boxes_prob_class = result_object.boxes.data.cpu().numpy()

    # 辺縁から指定されたピクセル内にあるバウンディングボックスを条件付きで除外
    filtered_boxes = []
    for box in bounding_boxes_prob_class:
        if (not is_top_edge and box[1] <= edge_margin) or \
           (not is_left_edge and box[0] <= edge_margin) or \
           (not is_bottom_edge and box[3] >= image_size - edge_margin) or \
           (not is_right_edge and box[2] >= image_size - edge_margin):
            continue
        filtered_boxes.append(box)

    # 除外後のバウンディングボックスを返却
    return np.array(filtered_boxes)

In [ ]:
def YOLO_NMS(selected_f, SIZE=640):
    Image.MAX_IMAGE_PIXELS = None
    
    try:
        img = np.array(Image.open(selected_f))
        height, width, _ = img.shape

        all_coordinates = []
        output_img_all_cord = img.copy()

        for window_size in window_sizes:
            stride = window_size[0] // 2

            # 左上からパッチを切り出す処理                    
            for y in range(0, height, stride):
                for x in range(0, width, stride):
                    x_end = min(width, x + window_size[0])
                    y_end = min(height, y + window_size[1])
                    image_patch = img[y:y_end, x:x_end]

                    # 実際のパッチサイズを計算
                    actual_patch_width = x_end - x
                    actual_patch_height = y_end - y

                    # スケーリング比率を計算
                    scale_x = actual_patch_width / SIZE
                    scale_y = actual_patch_height / SIZE

                    image_patch = Image.fromarray(image_patch).resize((SIZE, SIZE), Image.LANCZOS)
                    image_patch = np.array(image_patch)

                    is_top_edge = y == 0
                    is_left_edge = x == 0
                    predictions = yolo_predict(image_patch, is_top_edge=is_top_edge, is_left_edge=is_left_edge)

                    # 予測結果の処理
                    for pred in predictions:
                        x1, y1, x2, y2, score, cl = pred
                        x1 = (x1 * scale_x) + x
                        y1 = (y1 * scale_y) + y
                        x2 = (x2 * scale_x) + x
                        y2 = (y2 * scale_y) + y
                        all_coordinates.append([x1, y1, x2, y2, score, cl])
                        
            # 右端の処理
            for y in range(0, height - window_size[1] + 1, stride):
                x = width - window_size[0]  # 右端からパッチを切り出すためのX座標
                x_end = width  # 右端なので、x_endは画像の幅
                y_end = y + window_size[1]  # y_endは通常通り
                image_patch = img[y:y_end, x:x_end]  # パッチの切り出し

                # スケーリング比率を計算（修正）
                actual_patch_width = x_end - x  # 実際のパッチの幅
                scale_x = actual_patch_width / SIZE
                scale_y = window_size[1] / SIZE  # 高さは変わらない

                image_patch = Image.fromarray(image_patch).resize((SIZE, SIZE), Image.LANCZOS)
                image_patch = np.array(image_patch)

                is_right_edge = True
                predictions = yolo_predict(image_patch, is_right_edge=is_right_edge)

                # 予測結果の処理
                for pred in predictions:
                    x1, y1, x2, y2, score, cl = pred
                    x1 = (x1 * scale_x) + x  # xのオフセットを適用
                    y1 = (y1 * scale_y) + y
                    x2 = (x2 * scale_x) + x  # xのオフセットを適用
                    y2 = (y2 * scale_y) + y
                    all_coordinates.append([x1, y1, x2, y2, score, cl])
                     
            # 下端の処理
            for x in range(0, width - window_size[0] + 1, stride):
                y = height - window_size[1]  # 下端からパッチを切り出すためのY座標
                x_end = x + window_size[0]  # x_endは通常通り
                y_end = height  # 下端なので、y_endは画像の高さ
                image_patch = img[y:y_end, x:x_end]  # パッチの切り出し

                # スケーリング比率を計算（修正）
                actual_patch_height = y_end - y  # 実際のパッチの高さ
                scale_x = window_size[0] / SIZE  # 幅は変わらない
                scale_y = actual_patch_height / SIZE

                image_patch = Image.fromarray(image_patch).resize((SIZE, SIZE), Image.LANCZOS)
                image_patch = np.array(image_patch)

                is_bottom_edge = True
                predictions = yolo_predict(image_patch, is_bottom_edge=is_bottom_edge)

                # 予測結果の処理
                for pred in predictions:
                    x1, y1, x2, y2, score, cl = pred
                    x1 = (x1 * scale_x) + x
                    y1 = (y1 * scale_y) + y  # yのオフセットを適用
                    x2 = (x2 * scale_x) + x
                    y2 = (y2 * scale_y) + y  # yのオフセットを適用
                    all_coordinates.append([x1, y1, x2, y2, score, cl])

        # 予測されたすべてのバウンディングボックスを黒四角で描画
        for coord in all_coordinates:
            x1, y1, x2, y2, score, cl = map(int, coord)
            cv2.rectangle(output_img_all_cord, (x1, y1), (x2, y2), (0, 0, 0), 2)

        # 保存処理            
        all_coordinates_np = np.array(all_coordinates)
        np.save(os.path.splitext(selected_f)[0] + '_weight_'+ os.path.basename(os.path.splitext(mo)[0]) +'_0_bbox_coordinates.npy', all_coordinates_np)  
        cv2.imwrite(os.path.splitext(selected_f)[0] + '_'+ os.path.basename(os.path.splitext(mo)[0]) + '_0_output_image.png', 
                    np.array(Image.fromarray(output_img_all_cord).resize((output_img_all_cord.shape[1]//2, output_img_all_cord.shape[0]//2), 
                                                                         Image.LANCZOS)))
    
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"Error processing image {selected_f}: {str(e)}")

In [ ]:
%%time
window_sizes = [(2048, 2048), (1536, 1536), (1024, 1024)]

for mo in models:
    model = YOLO(mo)
    result = joblib.Parallel(n_jobs=16, verbose = 1)(joblib.delayed(YOLO_NMS)(selected_f) for selected_f in files)